In [ ]:
dataset_path = '/kaggle/input/eurosat-rgb/EuroSAT_RGB'


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
print(tf.__version__)


In [ ]:
train_dir = dataset_path
test_dir = dataset_path

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [ ]:
train_data = train_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


In [ ]:
test = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



In [ ]:
train_data.num_classes, train_data.class_indices

In [ ]:
class_names = list(train_data.class_indices.keys())
print(class_names)

## Data Visualization


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for images, labels in train_data:
    plt.imshow(images[0])
    plt.title(class_names[np.argmax(labels[0])])
    break

## Model

In [ ]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

#model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy()]
)

In [ ]:
model.fit(train_data, epochs=10, validation_data=test_data)

## Fine Tuning

In [ ]:
base_model.trainable = True
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(
    train_data,
    epochs=total_epochs,
    initial_epoch=EPOCHS,
    validation_data=test_data
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")